In [3]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging
from pykrige.kriging_tools import write_asc_grid
import pykrige.kriging_tools as kt
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Path, PathPatch
import os

In [20]:


def plot_ei(df,path_to_rep,species,date):
    df = df.dropna(axis = 0, how="any")
    lons = np.array(df.long)
    lats = np.array(df.lat)
    data = np.array(df.EI)
    grid_space = 0.05
    grid_lon = np.arange(lons.min()-0.05, lons.max()+0.1, grid_space) 
    grid_lat = np.arange(lats.min()-0.05, lats.max()+0.1, grid_space)
    OK = OrdinaryKriging(lons, lats, data, variogram_model='gaussian', verbose=True, enable_plotting=False,nlags=20)
    z1, ss1 = OK.execute('grid', grid_lon, grid_lat)
    xintrp, yintrp = np.meshgrid(grid_lon, grid_lat)
    
    ind = (z1>1)
    z1[ind] = 1
    ind = (z1<0)
    z1[ind] = 0
    
    fig, ax = plt.subplots(figsize=(10,10))
    m = Basemap(llcrnrlon=119.5,llcrnrlat=21.5,
                urcrnrlon=122.5,urcrnrlat=26, projection='merc', 
                resolution='i',area_thresh=1000.,ax=ax)
    m.drawcoastlines() #draw coastlines on the map
    
    x,y=m(xintrp,yintrp)
    lo,la = m(lons,lats)
    cs = ax.contourf(x,y,z1,
                       cmap="rainbow",
                       levels = np.linspace(0,1,11))
    #cs = ax.scatter(lo,la,c=data,cmap="rainbow")
    ax.scatter(lo,la,c=data,cmap="rainbow",vmin=0,vmax=1,edgecolor="k")
    cbar=m.colorbar(cs,location='right') #plot the colorbar on the map
    parallels = np.arange(21.5,26.0,1)
    m.drawparallels(parallels,labels=[1,0,0,0],fontsize=14, linewidth=0.0) #Draw the latitude labels on the map
    meridians = np.arange(119.5,123.5,1)
    m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=14, linewidth=0.0)

    x0,x1 = ax.get_xlim()
    y0,y1 = ax.get_ylim()
    map_edges = np.array([[x0,y0],[x1,y0],[x1,y1],[x0,y1]])
    polys = [p.boundary for p in m.landpolygons]
    polys = [map_edges]+polys[:]
    codes = [
    [Path.MOVETO]+[Path.LINETO for p in p[1:]]
    for p in polys
    ]

    polys_lin = [v for p in polys for v in p]
    codes_lin = [c for cs in codes for c in cs]

    path = Path(polys_lin, codes_lin)
    patch = PathPatch(path,facecolor='white', lw=0)
    ax.add_patch(patch)
#    plt.title("EI at {}".format(date))
    plt.savefig(path_to_rep  + "result/{}/{}_{}.png".format(species,date,species),
                bbox_inches="tight",dpi=150)

In [21]:
def digit00(x):
    if x < 10:
        return '0' + str(x)
    else:
        return str(x) 


def get_mi(prec,sm):
    shp = prec.shape
    mi = np.zeros(shp,dtype=np.float)

    for i in range(shp[0]):
        m = prec[i]
        if m < sm[0]:
            mi_index = 0
        elif m < sm[1]:
            mi_index = (m - sm[0])/(sm[1] - sm[0])
        elif m < sm[2]:
            mi_index = 1.
        elif m < sm[3]:
            mi_index = (m - sm[3])/(sm[2] - sm[3])
        else:
            mi_index = 0.
        mi[i] = mi_index

    return mi

def get_ti(td,tn,dv):
    shp = td.shape
    ti = np.zeros(shp,dtype=np.float)

    for i in range(shp[0]):
        hi_temp = td[i]
        low_temp = tn[i]
        dt = hi_temp - low_temp
        if dt > 0:
            iq = min( 1, 12 * max(0, hi_temp - dv[0]) ** 2 / dt / ((dv[1] - dv[0]) * 24))
        else:
            iq = np.nan
        ih = min( 1 - ( max(0, hi_temp - dv[2]) / (dv[3] - dv[2])) , 1)
        ih = max(0, ih)
        ti[i] = iq * ih
    return ti

def get_ei(path_to_rep,species,date):

  path_to_data = path_to_rep + 'analysis/'

  data_temp_max = pd.read_csv(path_to_data+'tmax.csv',encoding="UTF-8")
  data_temp_min = pd.read_csv(path_to_data+'tmin.csv',encoding="UTF-8")
  data_precp = pd.read_csv(path_to_data+'precp.csv',encoding="UTF-8")
  data_species = pd.read_csv(path_to_rep+'species.csv',encoding="UTF-8")

#  station_id = data_temp_max['id']
  temp_max = data_temp_max[['name',date]]
  temp_min = data_temp_min[['name',date]]
  precp = data_precp[['name',date]]

  ind = (data_species['scientific_name'] == species)
  sm, dv = np.zeros(4), np.zeros(4)
  for i in range(4):
    key = 'sm_' + '%1.1d' % i
    sm[i] =  data_species[key][ind]
    key = 'dv_' + '%1.1d' % i
    dv[i] =  data_species[key][ind]

  raw_data = temp_max.merge(temp_min,left_on='name',right_on='name')
  raw_data = raw_data.merge(precp,left_on='name',right_on='name')

  name = np.array(raw_data['name'])
  tmax = np.array(raw_data[date+'_x'])
  tmin = np.array(raw_data[date+'_y'])
  precp = np.array(raw_data[date])
  ind = (tmax>=-100) * (tmin>=-100) * (precp>=-100)

  name = name[ind]
  tmax = tmax[ind]
  tmin = tmin[ind]
  precp = precp[ind]

  ei = get_ti(tmax,tmin,dv) * get_mi(precp,sm)

  return name, ei

def run_ei(path_to_rep,species,date):
    name, ei = get_ei(path_to_rep,species,date)
    sta = pd.read_csv("../CWB_Stations_171226.csv")
    sta = sta[['id','long','lat']]
    eidt=pd.DataFrame(data={a:b for a,b in zip(name,ei)},index=['EI']).T
    eidt = eidt.merge(sta, left_index=True, right_on='id')
    return eidt



def main():
    global name
    global ei
    sp = pd.read_csv("../species.csv")
    spl = list(sp.scientific_name)
#    date = '2017-02'
    path_to_rep = "../"
#    species = 'achatina_fulica'
    for species in spl:
        path = path_to_rep + "result/{}".format(species)
        if not os.path.exists(path) or overwrite == True:
#            os.mkdir(path)
            for yy in range(2010,2018):
                for mm in range(1,13):
                    date = "{}-{}".format(yy,digit00(mm))
                    print(date)
            #            str         str      

                    name, ei = get_ei(path_to_rep,species,date)
                    eidt = run_ei(path_to_rep,species,date)
                    plot_ei(eidt,path_to_rep,species,date)
#            plt.show()
        else:
            print(path + " exist.")



In [22]:
if __name__ == '__main__':
    overwrite = True
    main()


2010-01
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.139408597411
Full Sill: 0.13940859745
Range: 0.189987552035
Nugget: 3.88308228391e-11 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...



/Users/nghianc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in greater_equal
/Users/nghianc/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


2010-02
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.135185224742
Full Sill: 0.179636030042
Range: 0.878962097041
Nugget: 0.0444508053004 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2010-03
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.10992270981
Full Sill: 0.156807117643
Range: 1.93950631999
Nugget: 0.0468844078328 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2010-04
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.154738940202
Full Sill: 0.154738940203
Range: 0.225887087377
Nugget: 8.7266034474e-13 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2010-05
Adjusting data for anisotropy.

2012-04
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.288070069986
Full Sill: 0.323872902551
Range: 2.36109593153
Nugget: 0.0358028325649 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2012-05
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.124398292415
Full Sill: 0.154256035586
Range: 4.32123717297
Nugget: 0.0298577431706 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2012-06
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.100028711469
Full Sill: 0.100028711469
Range: 4.32123717297
Nugget: 6.2675062322e-21 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2012-07
Adjusting data for anisotropy..

2014-06
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.0949218649276
Full Sill: 0.165571761605
Range: 0.75541717937
Nugget: 0.0706498966772 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2014-07
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.126467158335
Full Sill: 0.2673078729
Range: 4.32123717297
Nugget: 0.140840714565 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2014-08
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.23086351595
Full Sill: 0.360727669438
Range: 4.32123717297
Nugget: 0.129864153488 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2014-09
Adjusting data for anisotropy...
Ini

...

2016-03
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.187468044553
Full Sill: 0.233411980923
Range: 2.23426275786
Nugget: 0.0459439363704 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2016-04
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.245743289256
Full Sill: 0.300009947065
Range: 4.32920467966
Nugget: 0.0542666578087 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2016-05
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.0573613353332
Full Sill: 0.083151576412
Range: 0.734497004018
Nugget: 0.0257902410787 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2016-06
Adjusting data for anisotropy.

2010-05
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.024702375348
Full Sill: 0.074184632052
Range: 0.331646978803
Nugget: 0.049482256704 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2010-06
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.120706497426
Full Sill: 0.144635820845
Range: 4.32123717297
Nugget: 0.0239293234195 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2010-07
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.0408701199913
Full Sill: 0.0448377134632
Range: 0.797618072877
Nugget: 0.00396759347187 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2010-08
Adjusting data for anisotrop

2012-07
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.0541421279806
Full Sill: 0.0555664212119
Range: 0.490066344877
Nugget: 0.0014242932313 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2012-08
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.0322133269835
Full Sill: 0.0331149374343
Range: 4.32123717297
Nugget: 0.000901610450798 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2012-09
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.0298079865364
Full Sill: 0.0686371650058
Range: 1.23291998071
Nugget: 0.0388291784694 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2012-10
Adjusting data for anis

2014-09
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.0173114806621
Full Sill: 0.031033410976
Range: 0.538482506699
Nugget: 0.0137219303139 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2014-10
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.00806376148458
Full Sill: 0.0270407014333
Range: 0.195484837023
Nugget: 0.0189769399487 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2014-11
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.0688955923139
Full Sill: 0.102108479598
Range: 2.88295888264
Nugget: 0.0332128872838 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2014-12
Adjusting data for anisot

2016-11
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.0753112126493
Full Sill: 0.0911284030198
Range: 1.2164571821
Nugget: 0.0158171903705 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2016-12
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.0573263865776
Full Sill: 0.0687718924786
Range: 2.44264594902
Nugget: 0.011445505901 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2017-01
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'gaussian' Variogram Model
Partial Sill: 0.0556610611396
Full Sill: 0.0676148156156
Range: 2.72002395784
Nugget: 0.011953754476 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

2017-02
Adjusting data for anisotropy

In [7]:
global name
global ei
overwrite == True
sp = pd.read_csv("../species.csv")
spl = list(sp.scientific_name)
#    date = '2017-02'
path_to_rep = "../"
#    species = 'achatina_fulica'
for species in spl:
    path = path_to_rep + "output_csv/{}".format(species)
    if not os.path.exists(path) or overwrite == True:
#        os.mkdir(path)
        for yy in range(2010,2018):
            for mm in range(1,13):
                date = "{}-{}".format(yy,digit00(mm))
                print(date)
        #            str         str      

                name, ei = get_ei(path_to_rep,species,date)
                eidt = run_ei(path_to_rep,species,date)
                eidt.to_csv("{}/{}_{}.csv".format(path,date,species))
    else:
        print(path + " exist.")

species = list(pd.read_csv("../species.csv").scientific_name)
for sp in species:
    file = glob("{}/*".format(sp))
    print(sp)
    for i,f in enumerate(file):
        print(f)
        date = f.split('/')[1].split('_')[0]
        yy = date.split('-')[0]
        mm = date.split('-')[0]
        
        if i == 0:
            df = pd.read_csv(f)
            df['Time'] = [date] * len(df)
            df['Month'] = [mm] * len(df)
            df['Year'] = [yy] * len(df)
        else:
            dft = pd.read_csv(f)
            dft['Time'] = [date] * len(dft) 
            dft['Month'] = [mm] * len(dft)
            dft['Year'] = [yy] * len(dft)
            df = pd.concat([df,dft])
    df.to_csv("../output_csv/combine/{}.csv".format(sp))

2010-01
2010-02


/Users/nghianc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in greater_equal


2010-03
2010-04
2010-05
2010-06
2010-07
2010-08
2010-09
2010-10
2010-11
2010-12
2011-01
2011-02
2011-03
2011-04
2011-05
2011-06
2011-07
2011-08
2011-09
2011-10
2011-11
2011-12
2012-01
2012-02
2012-03
2012-04
2012-05
2012-06
2012-07
2012-08
2012-09
2012-10
2012-11
2012-12
2013-01
2013-02
2013-03
2013-04
2013-05
2013-06
2013-07
2013-08
2013-09
2013-10
2013-11
2013-12
2014-01
2014-02
2014-03
2014-04
2014-05
2014-06
2014-07
2014-08
2014-09
2014-10
2014-11
2014-12
2015-01
2015-02
2015-03
2015-04
2015-05
2015-06
2015-07
2015-08
2015-09
2015-10
2015-11
2015-12
2016-01
2016-02
2016-03
2016-04
2016-05
2016-06
2016-07
2016-08
2016-09
2016-10
2016-11
2016-12
2017-01
2017-02
2017-03
2017-04
2017-05
2017-06
2017-07
2017-08
2017-09
2017-10
2017-11
2017-12
2010-01
2010-02
2010-03
2010-04
2010-05
2010-06
2010-07
2010-08
2010-09
2010-10
2010-11
2010-12
2011-01
2011-02
2011-03
2011-04
2011-05
2011-06
2011-07
2011-08
2011-09
2011-10
2011-11
2011-12
2012-01
2012-02
2012-03
2012-04
2012-05
2012-06
2012-07
